In [3]:
import ujson as json

In [4]:
with open("asr_ranks.json", "r") as f:
    ranks_asr = json.load(f)

with open("caption_ranks.json", "r") as f:
    ranks_caption = json.load(f)

with open("image_ranks.json", "r") as f:
    ranks_image = json.load(f)

with open("cosmir_ranks.json", "r") as f:
    ranks_cosmir = json.load(f)

In [5]:
from data import queries

In [6]:
mrr_asr = []
mrr_caption = []
mrr_image = []
mrr_cosmir = []

In [13]:
for i in range(len(queries)):
    smallest_index = min([ranks_asr[i].index(q) for q in queries[i].correct_segments])

In [11]:
type(ranks_asr[1]), type(queries[1].correct_segments)

(list, list)